## DG-FONT

DG-FONT(https://paperswithcode.com/paper/dg-font-deformable-generative-networks-for) によるフォント生成です。

結果的には学習エポックを増やしても手書きの癖は再現されず、学習しすぎると生成画像が真っ白にもなってしまいました。

このような階層を想定しています。Colaboratoryを用いるので、ランタイムのタイプをGPUにします。

```
DG-FONT
|---Deformable-Convolution-V2-PyTorch-pytorch_1.0.0
|---DG-FONT-main
    |---handwriting
        |---id_0
            |---000.png
            |---001.png
            |---...
    |---save_folder
        |---id_0
            |---0000.png
            |---0001.png
            |---...
    |---results
        |---EXP_20211024-084920
            |---0_RNDEMA_1000_00.jpg
            |---...
    |---...

```

DG-FONT-mainフォルダ内のhandwriting下に、学習させたい手書き文字の画像を、save_folder下に変換したいフォントの画像を用意します。

今回はsave_folder下に明朝体の画像を用意し、明朝体フォントを手書き風に変換させようと試みました。


In [7]:
pwd

'/content/drive/Shareddrives/EEIC_AI'

In [8]:
ls

 2104.03064v2.gdoc
 2104.03064v2.pdf
 Deformable-Convolution-V2-PyTorch-pytorch_1.0.0/
 DG-Font.ipynb
 DG-Font-main/
 DG-Font-main-new/
 pytorch-CycleGAN-and-pix2pix-master/
 separate/
'Untitled1.ipynb のコピー'


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd ./drive/Shareddrives/EEIC_AI/

/content/drive/Shareddrives/EEIC_AI


Deformable-Convolution-V2-PyTorch-pytorch_1.0.0へ移動します。

In [12]:
cd Deformable-Convolution-V2-PyTorch-pytorch_1.0.0/

/content/drive/Shareddrives/EEIC_AI/Deformable-Convolution-V2-PyTorch-pytorch_1.0.0


DCN をインポートするために、"Deformable-Convolution-V2-PyTorch-pytorch_1.0.0"フォルダでこのセルを実行します。


In [13]:
!python setup.py build install

running build
running build_py
running build_ext
/usr/local/lib/python3.7/dist-packages/torch/utils/cpp_extension.py:370: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
running install
running bdist_egg
running egg_info
writing DCN.egg-info/PKG-INFO
writing dependency_links to DCN.egg-info/dependency_links.txt
writing top-level names to DCN.egg-info/top_level.txt
reading manifest file 'DCN.egg-info/SOURCES.txt'
writing manifest file 'DCN.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/functions
copying build/lib.linux-x86_64-3.7/functions/modulated_deform_conv_func.py -> build/bdist.linux-x86_64/egg/functions
copying build/lib.linux-x86_64-3.7/functions/deform_psroi_pooling_func.py -> build/bdist.linux-x86_64/

In [14]:
!pip3 install tensorboardX

     |████████████████████████████████| 124 kB 5.1 MB/s 


In [15]:
cd ../DG-Font-main/

/content/drive/Shareddrives/EEIC_AI/DG-Font-main


テスト用に明朝体の画像データを用意します。

character.txt に書かれた文字を、ttf_folder 下にある明朝体のttfファイルによって一文字ずつの画像データにして、それを save_folder に保存します。

In [16]:
!python font2img.py --ttf_path ttf_folder --chara character.txt --save_path save_folder --img_size 80 --chara_size 80

ttf_folder
1
ttf_folder/font_1_honokamin.ttf


### training

In [ ]:
# --load_model を設定することで途中から学習を再開させられます。
# 設定しない場合は始めから学習し、"logs"フォルダに学習フォルダが新規作成されます。
!python main.py --gpu 0 --img_size 80 --data_path handwriting --output_k 1  --epochs 200 --iters 100 --batch_size 8 --val_num 30 --val_batch 15 --load_model EXP_20211024-084920

PYTORCH VERSION 1.9.0+cu111
main.py:115: UserWarning: You have chosen a specific GPU. This will completely disable data parallelism.
  warnings.warn('You have chosen a specific GPU. This will completely '
False
False
MULTIPROCESSING DISTRIBUTED :  False
Use GPU: 0 for training
Init Generator
GENERATOR NF :  64
Init ContentEncoder
Traceback (most recent call last):
  File "main.py", line 403, in <module>
    main()
  File "main.py", line 170, in main
    main_worker(args.gpu, ngpus_per_node, args)
  File "main.py", line 206, in main_worker
    networks, opts = build_model(args)
  File "main.py", line 274, in build_model
    networks['G'] = Generator(args.img_size, args.sty_dim, use_sn=False)
  File "/content/drive/Shareddrives/EEIC_AI/DG-Font-main/models/generator.py", line 37, in __init__
    self.cnt_encoder = ContentEncoder(self.nf, n_downs, n_res, 'in', 'relu', 'reflect')
  File "/content/drive/Shareddrives/EEIC_AI/DG-Font-main/models/generator.py", line 115, in __init__
    self.dc

### test

In [17]:
!python main.py --gpu 0 --img_size 80 --data_path save_folder --output_k 1 --batch_size 8 --val_num 15 --val_batch 15 --validation --load_model EXP_20211024-084920

PYTORCH VERSION 1.9.0+cu111
main.py:115: UserWarning: You have chosen a specific GPU. This will completely disable data parallelism.
  warnings.warn('You have chosen a specific GPU. This will completely '
False
False
MULTIPROCESSING DISTRIBUTED :  False
Use GPU: 0 for training
Init Generator
GENERATOR NF :  64
Init ContentEncoder
Init Decoder
Init Generator
GENERATOR NF :  64
Init ContentEncoder
Init Decoder
=> loading checkpoint './logs/EXP_20211024-084920/model_40.ckpt'
=> loaded checkpoint './logs/EXP_20211024-084920/model_40.ckpt' (epoch 40)
USE CLASSES [0]
LABEL MAP: {0: 0}
MINIMUM DATA : 806
MAXIMUM DATA : 806
15
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the


DG-FONTのresultsのEXP_20211024-084920フォルダ内に学習過程の画像と画像の生成結果が保存されています。

0_EMA_**_00.jpgという画像が学習過程の画像です。

0_EMA_70_00.jpgは70エポック学習させたときの画像ですが、手書きの特徴画像は真っ白になりました。

0_EMA_1000_00.jpgは40エポックを終えたときの学習データをもとに生成した画像ですが、元の明朝体の文字が薄くかすれただけに見える画像となりました。

